In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from numpy import arange
import math

import seaborn as sns
sns.set_style("white")

import scipy as sp
from scipy import linalg, optimize

import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std

import collections
from collections import Counter

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.path as mpath
from matplotlib.dates import DateFormatter

import calendar
import datetime as dt
fromtimestamp = dt.datetime.fromtimestamp


In [3]:
# review ufo df
ufo = pd.read_csv('../data/ufo_clean.csv', index_col = 0)
ufo = ufo.rename(columns={'index':'ufo_id_num'})
ufo['ufo_id_num'] += 1
ufo.head(1)

,ufo_id_num,year_week,date_of_sight,year,month,week_of_year,state,city,sight_summary,ufo_shape
0,1,2006-52,2006-01-01,2006,1,52,NY,I-80 (unknown city proximity),"Traveling from Brooklyn, NY to Groton, NY on I...",Formation


In [3]:
# empty storages for google df data
googledfs = []
googledfnames = []
googledfdict = dict(zip(googledfnames,googledfs))

In [4]:
# trump import, rename, sorting, assign topic
trump = pd.read_csv('../Data/Raw/Trump.csv',header=1).sort_values('Week').rename(columns={'Donald Trump: (United States)':'rel_val'})
trump['topic']="trump"
googledfs.append(trump)
googledfnames.append('trump')
trump.head(1)

,Week,rel_val,topic
0,2016-05-01,14,trump


In [5]:
# obama import, rename, sorting, assign topic
obama = pd.read_csv('../Data/Raw/Obama.csv',header=1).sort_values('Week').rename(columns={'Barack Obama: (United States)':'rel_val'})
obama['topic']="obama"
googledfs.append(obama)
googledfnames.append('obama')
obama.head(1)

,Week,rel_val,topic
0,2008-05-04,9,obama


In [6]:
# kremlin  import, rename, sorting, assign topic
kremlin =  pd.read_csv('../Data/Raw/Kremlin.csv',header=1).sort_values('Week').rename(columns={'Kremlin: (United States)':'rel_val'})
kremlin['topic']="kremlin"
googledfs.append(kremlin)
googledfnames.append('kremlin')
kremlin.head(1)

,Week,rel_val,topic
0,2014-01-05,0,kremlin


In [7]:
# ebola import, rename, sorting, assign topic
ebola =  pd.read_csv('../Data/Raw/Ebola.csv',header=1).sort_values('Week').rename(columns={'Ebola: (United States)':'rel_val'})
ebola['topic']="ebola"
googledfs.append(ebola)
googledfnames.append('ebola')
ebola.head(1)

,Week,rel_val,topic
0,2014-04-06,1,ebola


In [8]:
# corona import, rename, sorting, assign topic
corona =  pd.read_csv('../Data/Raw/Corona.csv',header=1).sort_values('Week').rename(columns={'Coronavirus: (United States)':'rel_val'})
corona['topic']="corona"
googledfnames.append('corona')
googledfs.append(corona)
corona.head(1)

,Week,rel_val,topic
0,2019-09-01,<1,corona


In [9]:
# import arab spring, rename, sort, assign topic
arab = (pd.read_csv('../Data/Raw/arabs_2010_2011_googlesearch.csv')
        .rename(columns={'arab spring':'rel_val' ,'date' : 'Week'})
        .sort_values('Week'))
arab = arab.drop(['isPartial'], axis = 1)
arab['topic']="arab_spring"
googledfs.append(arab)
googledfnames.append('arab')
arab.head(1)

,Week,rel_val,topic
0,2010-01-10,0,arab_spring


In [10]:
# import boston marathon, rename, sort, assign topic
boston = (pd.read_csv('../Data/Raw/boston_marathon_weeks2013_googlesearch.csv')
            .rename(columns={'boston marathon bombings':'rel_val' ,'date' : 'Week'})
             .sort_values('Week'))
boston = boston.drop(['isPartial'], axis = 1)
boston['topic']="boston"
googledfs.append(boston)
googledfnames.append('boston')
boston.head(1)

,Week,rel_val,topic
0,2013-01-06,0,boston


In [11]:
# import financial crisis, rename, sort, assign topic
fcrisis = (pd.read_csv('../Data/Raw/fcrisis_weeks2008_googlesearch.csv')
           .rename(columns={'financial crisis':'rel_val' ,'date' : 'Week'})
           .sort_values('Week'))
fcrisis = fcrisis.drop(['isPartial'], axis = 1)
fcrisis['topic']="fcrisis"
googledfs.append(fcrisis)
googledfnames.append('fcrisis')
fcrisis.head(1)

,Week,rel_val,topic
0,2008-01-06,1,fcrisis


In [12]:
googledfs

[          Week  rel_val  topic
 0   2016-05-01       14  trump
 1   2016-05-08        9  trump
 2   2016-05-15        8  trump
 3   2016-05-22        8  trump
 4   2016-05-29        8  trump
 5   2016-06-05       10  trump
 6   2016-06-12       11  trump
 7   2016-06-19        9  trump
 8   2016-06-26        7  trump
 9   2016-07-03        8  trump
 10  2016-07-10       10  trump
 11  2016-07-17       24  trump
 12  2016-07-24       19  trump
 13  2016-07-31       19  trump
 14  2016-08-07       13  trump
 15  2016-08-14       12  trump
 16  2016-08-21       10  trump
 17  2016-08-28       11  trump
 18  2016-09-04       10  trump
 19  2016-09-11       11  trump
 20  2016-09-18       10  trump
 21  2016-09-25       22  trump
 22  2016-10-02       23  trump
 23  2016-10-09       39  trump
 24  2016-10-16       27  trump
 25  2016-10-23       21  trump
 26  2016-10-30       23  trump
 27  2016-11-06      100  trump
 28  2016-11-13       35  trump
 29  2016-11-20       19  trump
 30  201

In [13]:
# assign datetime col to all dfs
for df in googledfs:
    df['week_dt'] = pd.to_datetime(df['Week'])

In [14]:
# confirm
corona.head(1)

,Week,rel_val,topic,week_dt
0,2019-09-01,<1,corona,2019-09-01


In [15]:
# concat all dfs to "alldfs"
alldfs = pd.concat(googledfs).drop(columns='Week')

In [16]:
# assign cols "year","month","week_of_year"
alldfs['year'] = alldfs['week_dt'].dt.year.astype(str)
alldfs['month'] = alldfs['week_dt'].dt.month.astype(str)
alldfs['week_of_year'] = alldfs['week_dt'].dt.weekofyear

<ipython-input-16-300f796a80b8>:4: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  alldfs['week_of_year'] = alldfs['week_dt'].dt.weekofyear


In [17]:
#review
alldfs.head()

,rel_val,topic,week_dt,year,month,week_of_year
0,14,trump,2016-05-01,2016,5,17
1,9,trump,2016-05-08,2016,5,18
2,8,trump,2016-05-15,2016,5,19
3,8,trump,2016-05-22,2016,5,20
4,8,trump,2016-05-29,2016,5,21


In [18]:
# assign "week_id" col
alldfs['week_id'] = alldfs['year'] + "-" + alldfs['week_of_year'].astype(str)

In [19]:
# confirm
alldfs.head()

,rel_val,topic,week_dt,year,month,week_of_year,week_id
0,14,trump,2016-05-01,2016,5,17,2016-17
1,9,trump,2016-05-08,2016,5,18,2016-18
2,8,trump,2016-05-15,2016,5,19,2016-19
3,8,trump,2016-05-22,2016,5,20,2016-20
4,8,trump,2016-05-29,2016,5,21,2016-21


In [29]:
# changing <1 values in column rel val to 0.5 , changing to numeric dtype
alldfs['rel_val'] = np.where(alldfs['rel_val'] == '<1', 0.5 , alldfs['rel_val'])
alldfs['rel_val'] = pd.to_numeric(alldfs['rel_val'])
alldfs.dtypes

/Users/ulrikeanklam/opt/anaconda3/envs/matrix/lib/python3.8/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)


rel_val                float64
topic                   object
week_dt         datetime64[ns]
year                    object
month                   object
week_of_year             int64
week_id                 object
dtype: object

In [30]:
# export to csv, set to comment after export:
#alldfs.to_csv("../Data/Datasets/googletopics_weekid_v3.csv")